In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# experiment 1:
# first, creating a set of N randomly generated (m, 1) vectors:
import numpy as np
m = 5
N = 100
vectors = [np.random.rand(*(m, 1)) for _ in range(N)] # uniform distribution between 0 and 1
# print (vectors)

In [3]:
# finding nearest neighbor of each vector using O(N^2) brute force approach:
nearest_neighbors = {} # the keys will be the indices of the vectors from 0 to N-1, and the corresponding values will be the indices of its nearest neighbor
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors[i] = None
    for j in range(len(vectors)):
        if j != i:
            dist = np.linalg.norm(vectors[i] - vectors[j])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_neighbors[i] = j
print (nearest_neighbors) 

{0: 98, 1: 83, 2: 83, 3: 89, 4: 10, 5: 91, 6: 82, 7: 0, 8: 88, 9: 75, 10: 0, 11: 74, 12: 33, 13: 39, 14: 37, 15: 41, 16: 94, 17: 72, 18: 91, 19: 68, 20: 19, 21: 12, 22: 50, 23: 41, 24: 82, 25: 79, 26: 71, 27: 57, 28: 15, 29: 60, 30: 67, 31: 69, 32: 42, 33: 84, 34: 23, 35: 67, 36: 70, 37: 14, 38: 82, 39: 56, 40: 84, 41: 87, 42: 80, 43: 6, 44: 54, 45: 56, 46: 78, 47: 55, 48: 98, 49: 72, 50: 77, 51: 36, 52: 57, 53: 50, 54: 44, 55: 47, 56: 33, 57: 52, 58: 29, 59: 96, 60: 87, 61: 5, 62: 78, 63: 97, 64: 80, 65: 99, 66: 82, 67: 35, 68: 19, 69: 31, 70: 36, 71: 26, 72: 49, 73: 94, 74: 11, 75: 61, 76: 6, 77: 50, 78: 62, 79: 66, 80: 42, 81: 62, 82: 6, 83: 1, 84: 33, 85: 99, 86: 29, 87: 41, 88: 7, 89: 59, 90: 16, 91: 5, 92: 91, 93: 60, 94: 16, 95: 89, 96: 59, 97: 77, 98: 0, 99: 65}


In [4]:
# finding nearest neighbor of each vector using NaiveLSH:
from memristor.engine.model import NaiveLSH
from memristor.crossbar.model import LineResistanceCrossbar
from memristor.devices import StaticMemristor
naive_lsh = NaiveLSH(
    hash_size=3, # adjustable hyperparameter
    crossbar_class=LineResistanceCrossbar,
    crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
    memristor_model_class=StaticMemristor,
    memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
    m=m,
    r=1, # adjustable hyperparameter
)
reps = 5 # adjustable hyperparameter (repetitions of the hashing)
all_bins = []
for _ in range(reps):
    bins = {}
    for i in range(len(vectors)):
        hash = naive_lsh.inference(vectors[i])
        print (hash)
        if hash not in bins.keys():
            bins[hash] = [i]
        else:
            bins[hash].append(i)
    for bin in list(bins.values()):
        all_bins.append(bin)
#      {010:[1,5,7], 111:[5,6,7]}

# now at this point all_bins is a list like [[1,2], [1,3,5], ... ] where each element of all_bins is a bin containing indices of vectors that are likely
# to be close to each other. so now to find the nearest neighbor for each vector, we simply iterate through and check only those vectors that share a bin
# with it, so in this case for 1 we would check 2, 3, and 5, to find the nearest neighbor
nearest_neighbors_approx = {}
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors_approx[i] = None
    for bin in all_bins:
        if i in bin:
            for j in bin:
                if j != i:
                    dist = np.linalg.norm(vectors[i] - vectors[j])
                    if dist < nearest_dist:
                        nearest_dist = dist 
                        nearest_neighbors_approx[i] = j
print (nearest_neighbors_approx)

c:\Users\Ahsan\Desktop\pytorch-metal-oxide-memristor-crossbar\memristor\crossbar\model.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  self.fitted_w = torch.tensor([[self.memristors[i][j].g_linfit for j in range(ideal_w.shape[1])]


011
111
111
011
011
011
011
011
011
011
011
111
011
011
011
111
111
111
111
011
011
011
111
111
111
011
011
111
111
011
011
111
111
011
111
111
111
011
011
011
011
111
111
111
011
011
111
111
011
011
111
111
111
111
011
111
011
111
011
111
011
010
011
011
011
011
011
111
011
111
111
011
011
111
111
010
011
011
111
011
011
111
011
111
011
011
111
011
011
111
111
011
011
011
111
111
111
011
011
011
011
111
111
011
011
011
011
011
011
011
011
111
011
011
011
111
111
111
111
011
011
011
111
111
111
011
011
111
111
011
011
111
111
011
111
111
111
011
011
011
011
111
111
111
011
011
111
111
011
011
111
111
111
111
011
111
011
111
011
111
011
010
011
011
011
011
011
111
011
111
111
011
011
111
111
010
011
011
111
011
011
111
011
111
011
011
111
011
011
111
111
011
011
011
111
111
111
011
011
011
011
111
111
011
011
011
011
011
011
011
011
111
011
011
011
111
111
111
111
011
011
011
111
111
111
011
011
111
111
011
011
111
111
011
111
111
111
011
011
011
011
111
111
111
011
011
111
111
011
011


In [5]:
print (nearest_neighbors)
print (nearest_neighbors_approx)

{0: 98, 1: 83, 2: 83, 3: 89, 4: 10, 5: 91, 6: 82, 7: 0, 8: 88, 9: 75, 10: 0, 11: 74, 12: 33, 13: 39, 14: 37, 15: 41, 16: 94, 17: 72, 18: 91, 19: 68, 20: 19, 21: 12, 22: 50, 23: 41, 24: 82, 25: 79, 26: 71, 27: 57, 28: 15, 29: 60, 30: 67, 31: 69, 32: 42, 33: 84, 34: 23, 35: 67, 36: 70, 37: 14, 38: 82, 39: 56, 40: 84, 41: 87, 42: 80, 43: 6, 44: 54, 45: 56, 46: 78, 47: 55, 48: 98, 49: 72, 50: 77, 51: 36, 52: 57, 53: 50, 54: 44, 55: 47, 56: 33, 57: 52, 58: 29, 59: 96, 60: 87, 61: 5, 62: 78, 63: 97, 64: 80, 65: 99, 66: 82, 67: 35, 68: 19, 69: 31, 70: 36, 71: 26, 72: 49, 73: 94, 74: 11, 75: 61, 76: 6, 77: 50, 78: 62, 79: 66, 80: 42, 81: 62, 82: 6, 83: 1, 84: 33, 85: 99, 86: 29, 87: 41, 88: 7, 89: 59, 90: 16, 91: 5, 92: 91, 93: 60, 94: 16, 95: 89, 96: 59, 97: 77, 98: 0, 99: 65}
{0: 98, 1: 83, 2: 83, 3: 88, 4: 10, 5: 91, 6: 82, 7: 0, 8: 88, 9: 92, 10: 0, 11: 74, 12: 33, 13: 39, 14: 37, 15: 41, 16: 94, 17: 22, 18: 59, 19: 68, 20: 19, 21: 12, 22: 50, 23: 41, 24: 1, 25: 79, 26: 71, 27: 57, 28: 15,

In [6]:
nearest_neighbors == nearest_neighbors_approx

False

In [8]:
print (len(all_bins))
print (all_bins)
print ([len(bin) for bin in all_bins])

15
[[0, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 19, 20, 21, 25, 26, 29, 30, 33, 37, 38, 39, 40, 44, 45, 48, 49, 54, 56, 58, 60, 62, 63, 64, 65, 66, 68, 71, 72, 76, 77, 79, 80, 82, 84, 85, 87, 88, 91, 92, 93, 97, 98, 99], [1, 2, 11, 15, 16, 17, 18, 22, 23, 24, 27, 28, 31, 32, 34, 35, 36, 41, 42, 43, 46, 47, 50, 51, 52, 53, 55, 57, 59, 67, 69, 70, 73, 74, 78, 81, 83, 86, 89, 90, 94, 95, 96], [61, 75], [0, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 19, 20, 21, 25, 26, 29, 30, 33, 37, 38, 39, 40, 44, 45, 48, 49, 54, 56, 58, 60, 62, 63, 64, 65, 66, 68, 71, 72, 76, 77, 79, 80, 82, 84, 85, 87, 88, 91, 92, 93, 97, 98, 99], [1, 2, 11, 15, 16, 17, 18, 22, 23, 24, 27, 28, 31, 32, 34, 35, 36, 41, 42, 43, 46, 47, 50, 51, 52, 53, 55, 57, 59, 67, 69, 70, 73, 74, 78, 81, 83, 86, 89, 90, 94, 95, 96], [61, 75], [0, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 19, 20, 21, 25, 26, 29, 30, 33, 37, 38, 39, 40, 44, 45, 48, 49, 54, 56, 58, 60, 62, 63, 64, 65, 66, 68, 71, 72, 76, 77, 79, 80, 82, 84, 85, 87, 88, 91, 92, 93, 97,

In [ ]:
# when experimenting with like a bigger dataset and stuff come up with  a metric to compare these 2 dicts
# also compare the runtime complexities, cuz its possible that its working so well because of sth wrong in the implementation whiich results in the runtime
# just being the same as the brute force method

In [9]:
'''
things to check: 
- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity
- varying the parameters like N, m, hash size, etc. (try bigger/more data)
- using lineres_memristive_vmm not naive_memristive_vmm
- experiment with varying non-idealities
- experiment 2, create visualizations
- write section 4 of the paper    (rn highest priority is finalizing the experiment and section 4 of the paper)
- is the change made in StaticMemristor fine?
'''

'\nthings to check: \n- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity\n- varying the parameters like N, m, hash size, etc. (try bigger/more data)\n- using lineres_memristive_vmm not naive_memristive_vmm\n'